# Q6 Batch Norm and SELU (4.5 Points)
Generate training and test datasets for a binary classiﬁcation problem using Fashion-MNIST with class 1 being a combination of sneaker and pullover and class 0 being the combination of sandal and shirt categories. 
- Train the model using Logistic regression (No Hidden Layers). Report train and test loss.
- Train a Neural Network with one hidden layer (100 neurons). Use Adam optimizer and Relu activation for hidden layer.  First overfit a small sample to check errors and get idea of learning rate. Then train on complete dataset. Add regularization (dropout or weight decay)if needed.
- Now add another hidden layer (50 Neurons). Adjust the learning rate if you have to. Add regularization (dropout or weight decay) if needed.
- Now try adding Batch Normalization and compare the train and test loss : Is it converging faster than before? Does it produce a better model? How does it affect training speed? **Do not use dropout with batch normalization.**
- Try replacing Batch Normalization with SELU, and make the necessary adjustments to ensure the network self-normalizes (i.e., standardize the input features, use LeCun normal initialization, make sure the DNN contains only a sequence of dense layers). Compare the results with Batch Normalization. **For SELU if you are using dropout then use alpha dropout.** Alpha dropout make sure that network is self normalized.


# <Font color = 'pickle'>**Importing Libraries**

In [1]:
if 'google.colab' in str(get_ipython()):
  print('Running on Colab')
else:
  print('Not running on Colab')

Running on Colab


In [2]:
if 'google.colab' in str(get_ipython()):
  !pip install wandb --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 19.2 MB/s 
     |████████████████████████████████| 166 kB 74.2 MB/s 
     |████████████████████████████████| 182 kB 60.8 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 166 kB 65.7 MB/s 
     |████████████████████████████████| 162 kB 28.5 MB/s 
     |████████████████████████████████| 162 kB 81.1 MB/s 
     |████████████████████████████████| 158 kB 79.9 MB/s 
     |████████████████████████████████| 157 kB 77.2 MB/s 
     |████████████████████████████████| 157 kB 70.7 MB/s 
     |████████████████████████████████| 157 kB 63.5 MB/s 
     |████████████████████████████████| 157 kB 73.7 MB/s 
     |████████████████████████████████| 157 kB 64.2 MB/s 
     |████████████████████████████████| 157 kB 64.8 MB/s 
     |████████████████████████████████| 157 kB 68.6 MB/s 
     |███████████████████████████

In [3]:
# mount google drive
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Importing the necessary libraries
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

from torch.optim.lr_scheduler import ReduceLROnPlateau, ExponentialLR, CyclicLR, OneCycleLR, StepLR

import numpy as np
import random

from datetime import datetime
from pathlib import Path
import sys
from types import SimpleNamespace

import wandb

# <Font color = 'pickle'>**Initializing Wandb**

In [5]:
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
wandb.init(name = "Hw5_FASHION_MNIST.ipynb", project = 'Deep_Learning_Class_UTD')

wandb: Currently logged in as: pranavshekhar2. Use `wandb login --relogin` to force relogin


# Q6 Batch Norm and SELU (4.5 Points)
Generate training and test datasets for a binary classiﬁcation problem using Fashion-MNIST with class 1 being a combination of sneaker and pullover and class 0 being the combination of sandal and shirt categories. 
- Train the model using Logistic regression (No Hidden Layers). Report train and test loss.
- Train a Neural Network with one hidden layer (100 neurons). Use Adam optimizer and Relu activation for hidden layer.  First overfit a small sample to check errors and get idea of learning rate. Then train on complete dataset. Add regularization (dropout or weight decay)if needed.
- Now add another hidden layer (50 Neurons). Adjust the learning rate if you have to. Add regularization (dropout or weight decay) if needed.
- Now try adding Batch Normalization and compare the train and test loss : Is it converging faster than before? Does it produce a better model? How does it affect training speed? **Do not use dropout with batch normalization.**
- Try replacing Batch Normalization with SELU, and make the necessary adjustments to ensure the network self-normalizes (i.e., standardize the input features, use LeCun normal initialization, make sure the DNN contains only a sequence of dense layers). Compare the results with Batch Normalization. **For SELU if you are using dropout then use alpha dropout.** Alpha dropout make sure that network is self normalized.


In [7]:
# This is the path where we will downlaod and save data
if 'google.colab' in str(get_ipython()):
    data_folder = Path('/content/drive/MyDrive/Deep_Learning_UTD/Dataset')
    model_folder = Path('/content/drive/MyDrive/Deep_Learning_UTD/Model')
else:
    data_folder = Path('/home/harpreet/Insync/google_drive_shaannoor/data/datasets')

## <Font color = 'pickle'>**Train and Test Dataset - FASHION_MNIST**

In [8]:
# Transform to convert images to pytorch tensors
trans1 = transforms.ToTensor()

# Transform to normalize the data
# The mean and std are based on train subset which we will create below
trans2 = transforms.Normalize((0.2857,), (0.3528))
trans = transforms.Compose([trans1, trans2])

# Download the training_validation data (we will create two subsets - trainset and valset frpm this)
train_val_set = torchvision.datasets.FashionMNIST(root = data_folder, 
                                             train = True, 
                                             transform = trans, 
                                             download = True)

# Download the testing data
testset = torchvision.datasets.FashionMNIST(root = data_folder, 
                                            train = False, 
                                            transform = trans, 
                                            download = True)

In [18]:
def split_dataset(base_dataset, fraction, seed):
    split_a_size = int(fraction * len(base_dataset))
    split_b_size = len(base_dataset) - split_a_size
    return torch.utils.data.random_split(base_dataset, [split_a_size, split_b_size], generator=torch.Generator().manual_seed(seed)
    )

In [19]:
trainset, validset = split_dataset(train_val_set,0.8,10)

In [20]:
#DataLoaders

# Initializing the batch size
batch_size = 256

# Creating data loader for train set
train_loader = torch.utils.data.DataLoader(dataset= trainset,
                                           batch_size = batch_size,
                                           shuffle = True)

valid_loader = torch.utils.data.DataLoader(dataset = validset,
                                           batch_size = batch_size,
                                           shuffle = False)

# Creating data loader for test set
test_loader = torch.utils.data.DataLoader(dataset = testset,
                                          batch_size = batch_size,
                                          shuffle = False)

In [10]:
#Checking the mapping of the labels

train_val_set.data
train_val_set.class_to_idx

{'T-shirt/top': 0,
 'Trouser': 1,
 'Pullover': 2,
 'Dress': 3,
 'Coat': 4,
 'Sandal': 5,
 'Shirt': 6,
 'Sneaker': 7,
 'Bag': 8,
 'Ankle boot': 9}

In [11]:
#Creating a subset of data consisting of class 1 and class 0

from torch.utils.data.dataset import Subset

#Sneaker and Pullover - Class1
train_idx_1 = np.where((train_val_set.targets==7) | (train_val_set.targets==2))[0]
train_subset_1 = Subset(train_val_set,train_idx_1)


#Sandal and Shirt - Class 0
train_idx_2 = np.where((train_val_set.targets==5) | (train_val_set.targets==6))[0]
train_subset_2 = Subset(train_val_set,train_idx_2)


In [12]:
#Sneaker and Pullover - Class 1 - Creating labels for Class 1 Group

train_subset_1.dataset.data[train_subset_1.indices].shape
labels_subset_1 = [1]*(train_subset_1.dataset.data[train_subset_1.indices].shape[0]) #Label 1 is created here


In [13]:
train_val_set.targets[train_subset_1.indices]

tensor([2, 7, 2,  ..., 2, 7, 2])

In [14]:
#Sandal and Shirt - Class 0 - Creating labels for Class 0 Group

train_subset_2.dataset.data[train_subset_2.indices].shape
labels_subset_2 = [0]*(train_subset_2.dataset.data[train_subset_2.indices].shape[0])

In [15]:
train_val_set.targets[train_subset_2.indices]

tensor([5, 5, 5,  ..., 6, 5, 5])

In [16]:
train_data1 = []
for i in range(len(train_subset_1)):
   train_data1.append([train_subset_1[i], labels_subset_1[i]])

train_data2 = []
for j in range(len(train_subset_2)):
   train_data2.append([train_subset_2[j], labels_subset_2[j]])

#Combined Dataset
train_data1.extend(train_data2)

# Class for Model

In [17]:
class FashionMNIST_NN(nn.Module):
  def __init__(self, input_dim, output_dim, h_sizes, dprob, non_linearity, batch_norm):
    super().__init__()
    self.input_dim = input_dim
    self.h_sizes = h_sizes # list of hidden sizes
    self.non_linearity = non_linearity 
    self.batch_norm = batch_norm
    self.dprob = dprob # list of dropout probabilities
    self.output_dim = output_dim

    # Initialize hidden layers  
    model_layers = [nn.Flatten()]

    # hidden layers
    for i, hidden_size in enumerate(self.h_sizes):
      model_layers.append(nn.Linear(input_dim, hidden_size))
      model_layers.append(self.non_linearity)
      model_layers.append(nn.Dropout(p=dprob[i]))

      if self.batch_norm:
        model_layers.append(nn.BatchNorm1d(hidden_size, momentum=0.9))
      
      input_dim = hidden_size

    # output layer  
    model_layers.append(nn.Linear(self.h_sizes[-1], self.output_dim))

    self.module_list = nn.ModuleList(model_layers)


  def forward(self, x):
    for layer in self.module_list:
      x = layer(x)

    # we are not using softmax function in the forward passs
    # nn.crossentropy loss (which we will use to define our loss) combines  nn.LogSoftmax() and nn.NLLLoss() in one single class
    return x  

In [33]:
def train(train_loader, loss_function, model, optimizer, grad_clipping, max_norm, log_batch, log_interval):

  # Training Loop 

  # initilalize variables as global
  # these counts will be updated every epoch
  global batch_ct_train

  # Initialize train_loss at the he start of the epoch
  running_train_loss = 0
  running_train_correct = 0
  
  # put the model in training mode

  model.train()
  # Iterate on batches from the dataset using train_loader
  for input_, targets in train_loader:
    
    # move inputs and outputs to GPUs
    input_ = input_.to(device)
    targets = targets.to(device)


    # Step 1: Forward Pass: Compute model's predictions 
    output = model(input_)
    
    # Step 2: Compute loss
    loss = loss_function(output, targets)

    # Correct prediction
    y_pred = torch.argmax(output, dim = 1)
    correct = torch.sum(y_pred == targets)

    batch_ct_train += 1

    # Step 3: Backward pass -Compute the gradients
    optimizer.zero_grad()
    loss.backward()

    # Gradient Clipping
    if grad_clipping:
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm, norm_type=2)

    # Step 4: Update the parameters
    optimizer.step()
          
    # Add train loss of a batch 
    running_train_loss += loss.item()

    # Add Corect counts of a batch
    running_train_correct += correct

    # log batch loss and accuracy
    if log_batch:
      if ((batch_ct_train + 1) % log_interval) == 0:
        wandb.log({f"Train Batch Loss  :": loss})
        wandb.log({f"Train Batch Acc :": correct/len(targets)})

  
  # Calculate mean train loss for the whole dataset for a particular epoch
  train_loss = running_train_loss/len(train_loader)

  # Calculate accuracy for the whole dataset for a particular epoch
  train_acc = running_train_correct/len(train_loader.dataset)
  

  return train_loss, train_acc

In [34]:
def validate(valid_loader, loss_function, model, log_batch, log_interval):

  # initilalize variables as global
  # these counts will be updated every epoch
  global batch_ct_valid

  # Validation/Test loop
  # Initialize valid_loss at the he strat of the epoch
  running_val_loss = 0
  running_val_correct = 0

  # put the model in evaluation mode
  model.eval()

  with torch.no_grad():
    for input_,targets in valid_loader:

      # move inputs and outputs to GPUs
      input_ = input_.to(device)
      targets = targets.to(device)

      # Step 1: Forward Pass: Compute model's predictions 
      output = model(input_)

      # Step 2: Compute loss
      loss = loss_function(output, targets)

      # Correct Predictions
      y_pred = torch.argmax(output, dim = 1)
      correct = torch.sum(y_pred == targets)

      batch_ct_valid += 1

      # Add val loss of a batch 
      running_val_loss += loss.item()

      # Add correct count for each batch
      running_val_correct += correct

      # log batch loss and accuracy
      if log_batch:
        if ((batch_ct_valid + 1) % log_interval) == 0:
          wandb.log({f"Valid Batch Loss  :": loss})
          wandb.log({f"Valid Batch Accuracy :": correct/len(targets)})

    # Calculate mean val loss for the whole dataset for a particular epoch
    val_loss = running_val_loss/len(valid_loader)

    # Calculate accuracy for the whole dataset for a particular epoch
    val_acc = running_val_correct/len(valid_loader.dataset)

    # scheduler step
    # scheduler.step(valid_loss)
    # scheduler.step()
    
  return val_loss, val_acc

In [35]:
def train_loop(train_loader, valid_loader, model, optimizer, loss_function, epochs, device, patience, early_stopping,
               file_model):
    
  """ 
  Function for training the model and plotting the graph for train & validation loss vs epoch.
  Input: iterator for train dataset, initial weights and bias, epochs, learning rate, batch size.
  Output: final weights, bias and train loss and validation loss for each epoch.
  """

  # Create lists to store train and val loss at each epoch
  train_loss_history = []
  valid_loss_history = []
  train_acc_history = []
  valid_acc_history = []

  # initialize variables for early stopping

  delta = 0
  best_score = None
  valid_loss_min = np.Inf
  counter_early_stop=0
  early_stop=False

  # Iterate for the given number of epochs
  # Step 5: Repeat steps 1 - 4

  for epoch in range(epochs):

    t0 = datetime.now()

    # Get train loss and accuracy for one epoch
    train_loss, train_acc = train(train_loader, loss_function, model, optimizer, 
                                  wandb.config.grad_clipping, wandb.config.max_norm,
                                  wandb.config.log_batch, wandb.config.log_interval)
    valid_loss, valid_acc   = validate(valid_loader, loss_function, model, wandb.config.log_batch, wandb.config.log_interval)

    dt = datetime.now() - t0

    # Save history of the Losses and accuracy
    train_loss_history.append(train_loss)
    train_acc_history.append(train_acc)

    valid_loss_history.append(valid_loss)
    valid_acc_history.append(valid_acc)

    # Log the train and valid loss to wandb
    wandb.log({f"Train Loss :": train_loss, "epoch": epoch})
    wandb.log({f"Train Acc :": train_acc, "epoch": epoch})

    wandb.log({f"Valid Loss :": valid_loss, "epoch": epoch})
    wandb.log({f"Valid Acc :": valid_acc, "epoch": epoch})

    if early_stopping:
      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        counter_early_stop += 1
        print(f'Early stoping counter: {counter_early_stop} out of {patience}')
        if counter_early_stop > patience:
          early_stop = True

      
      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        counter_early_stop=0
        valid_loss_min = valid_loss

      if early_stop:
        print('Early Stopping')
        break

    else:

      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        print(f'Validation loss has not decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Not Saving Model...')
      
      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss
    
    # Print the train loss and accuracy for given number of epochs, batch size and number of samples
    print(f'Epoch : {epoch+1} / {epochs}')
    print(f'Time to complete {epoch+1} is {dt}')
    # print(f'Learning rate: {scheduler._last_lr[0]}')
    print(f'Train Loss: {train_loss : .4f} | Train Accuracy: {train_acc * 100 : .4f}%')
    print(f'Valid Loss: {valid_loss : .4f} | Valid Accuracy: {valid_acc * 100 : .4f}%')
    print()
    torch.cuda.empty_cache()

  return train_loss_history, train_acc_history, valid_loss_history, valid_acc_history

In [36]:
def get_acc_pred(data_loader, model, device):
    
  """ 
  Function to get predictions and accuracy for a given data using estimated model
  Input: Data iterator, Final estimated weoights, bias
  Output: Prections and Accuracy for given dataset
  """

  # Array to store predicted labels
  predictions = torch.Tensor() # empty tensor
  predictions = predictions.to(device) # move predictions to GPU

  # Array to store actual labels
  y = torch.Tensor() # empty tensor
  y = y.to(device)

  # put the model in evaluation mode
  model.eval()
  
  # Iterate over batches from data iterator
  with torch.no_grad():
    for input_, targets in data_loader:
      
      # move inputs and outputs to GPUs
      
      input_ = input_.to(device)
      targets = targets.to(device)
      
      # Calculated the predicted labels
      output = model(input_)

      # Choose the label with maximum probability
      prediction = torch.argmax(output, dim = 1)

      # Add the predicted labels to the array
      predictions = torch.cat((predictions, prediction)) 

      # Add the actual labels to the array
      y = torch.cat((y, targets)) 

  # Check for complete dataset if actual and predicted labels are same or not
  # Calculate accuracy
  acc = (predictions == y).float().mean()

  # Return tuple containing predictions and accuracy
  return predictions, acc  

# Logistic Regression

In [21]:
class LogisticRegression(torch.nn.Module):
     def __init__(self, input_dim, output_dim):
         super(LogisticRegression, self).__init__()
         self.linear = torch.nn.Linear(input_dim, output_dim)
     def forward(self, x):
         outputs = torch.sigmoid(self.linear(x))
         return outputs

In [22]:
epochs = 200
input_dim = 3*32*32 # Two inputs x1 and x2 
output_dim = 1 # Single binary output 
learning_rate = 0.01

In [23]:
model = LogisticRegression(input_dim,output_dim)

In [24]:
criterion = torch.nn.BCELoss()

In [25]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# One Hidden Layer

In [41]:
hyperparameters1 = SimpleNamespace(
    epochs = 20,
    input_dim = 28*28,
    output_dim = 10,
    h_sizes = [100]  , #  1 Hidden Layer of 100 Neurons
    dprob = [0],
    non_linearity = nn.ReLU(),
    batch_norm = False,
    batch_size=25,
    learning_rate=0.07,
    dataset="FASHION_MNIST",
    architecture="MLP",
    log_interval = 1,
    log_batch = True,
    file_model = model_folder/'exp1_overfit_mnist.pt',
    grad_clipping = False, # DO NOT CHANGE hyperparameters below this 
    early_stopping = False,
    max_norm = 1,
    momentum = 0,
    patience = 3,
    # scheduler_factor = 0.5,
    # scheduler_patience = 0,
    weight_decay = 0.00
    )

In [42]:
wandb.config = hyperparameters1
wandb.config

namespace(architecture='MLP', batch_norm=False, batch_size=25, dataset='FASHION_MNIST', dprob=[0], early_stopping=False, epochs=20, file_model=PosixPath('/content/drive/MyDrive/Deep_Learning_UTD/Model/exp1_overfit_mnist.pt'), grad_clipping=False, h_sizes=[100], input_dim=784, learning_rate=0.07, log_batch=True, log_interval=1, max_norm=1, momentum=0, non_linearity=ReLU(), output_dim=10, patience=3, weight_decay=0.0)

In [43]:
# Fix seed value
SEED = 2344
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Data Loader
train_loader = torch.utils.data.DataLoader(trainset, batch_size=wandb.config.batch_size, shuffle = True)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=wandb.config.batch_size, shuffle = False)
# test_loader = torch.utils.data.DataLoader(testset, batch_size=wandb.config.batch_size,   shuffle = False)

# device 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

wandb.config.device = device

model = FashionMNIST_NN(wandb.config.input_dim, wandb.config.output_dim, wandb.config.h_sizes, 
                          wandb.config.dprob, wandb.config.non_linearity, wandb.config.batch_norm)

# Initialize weights from normal distribution with mean 0 and standard deviation 0.01
def init_weights(layer):
  if type(layer) == nn.Linear:
    torch.nn.init.kaiming_normal_(layer.weight, mean = 0, std = 0.01)
    # torch.nn.init.normal_(layer.weight, mean = 0, std = 0.001)
    torch.nn.init.zeros_(layer.bias)

model.to(wandb.config.device)
# model.apply(init_weights)

# loss_function
loss_function = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), 
                            lr = wandb.config.learning_rate, 
                            weight_decay = wandb.config.weight_decay)

# scheduler = ReduceLROnPlateau(optimizer, mode='min', factor= wandb.config.scheduler_factor, 
#                              patience=wandb.config.scheduler_patience, verbose=True)

#scheduler = StepLR(optimizer, gamma=0.4,step_size=1, verbose=True)

In [44]:
# See live graphs in the notebook.
#%%wandb 
batch_ct_train, batch_ct_valid = 0, 0
train_loss_history, train_acc_history, valid_loss_history, valid_acc_history = train_loop(train_loader, 
                                                                                          valid_loader, 
                                                                                          model, 
                                                                                          optimizer, 
                                                                                          loss_function, 
                                                                                          wandb.config.epochs, 
                                                                                          wandb.config.device,
                                                                                          wandb.config.patience,
                                                                                          wandb.config.early_stopping,
                                                                                          wandb.config.file_model,
                                                                                          )

Validation loss has decreased (inf --> 2.400696). Saving Model...
Epoch : 1 / 20
Time to complete 1 is 0:00:13.682159
Train Loss:  2.6776 | Train Accuracy:  23.7604%
Valid Loss:  2.4007 | Valid Accuracy:  11.1750%

Validation loss has decreased (2.400696 --> 2.399744). Saving model...
Epoch : 2 / 20
Time to complete 2 is 0:00:13.766537
Train Loss:  2.8962 | Train Accuracy:  12.6042%
Valid Loss:  2.3997 | Valid Accuracy:  11.3500%

Validation loss has decreased (2.399744 --> 2.296860). Saving model...
Epoch : 3 / 20
Time to complete 3 is 0:00:13.968270
Train Loss:  2.6764 | Train Accuracy:  12.2667%
Valid Loss:  2.2969 | Valid Accuracy:  10.8417%

Validation loss has decreased (2.296860 --> 2.292094). Saving model...
Epoch : 4 / 20
Time to complete 4 is 0:00:13.827049
Train Loss:  2.3503 | Train Accuracy:  10.7208%
Valid Loss:  2.2921 | Valid Accuracy:  11.1250%

Validation loss has not decreased (2.292094 --> 2.376915). Not Saving Model...
Epoch : 5 / 20
Time to complete 5 is 0:00:16.3

In [ ]:
hyperparameters2 = SimpleNamespace(
    epochs = 20,
    input_dim = 28*28,
    output_dim = 10,
    h_sizes = [100 ,50]  , #  2 Hidden Layer of 100,50 Neurons
    dprob = [0]*2,
    non_linearity = nn.ReLU(),
    batch_norm = False,
    batch_size=25,
    learning_rate=0.07,
    dataset="FASHION_MNIST",
    architecture="MLP",
    log_interval = 1,
    log_batch = True,
    file_model = model_folder/'exp1_overfit_mnist.pt',
    grad_clipping = False, # DO NOT CHANGE hyperparameters below this 
    early_stopping = False,
    max_norm = 1,
    momentum = 0,
    patience = 3,
    # scheduler_factor = 0.5,
    # scheduler_patience = 0,
    weight_decay = 0.00
    )

In [ ]:
wandb.config = hyperparameters2
wandb.config

In [ ]:
# Fix seed value
SEED = 2344
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Data Loader
train_loader = torch.utils.data.DataLoader(trainset, batch_size=wandb.config.batch_size, shuffle = True)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=wandb.config.batch_size, shuffle = False)
# test_loader = torch.utils.data.DataLoader(testset, batch_size=wandb.config.batch_size,   shuffle = False)

# device 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

wandb.config.device = device

model2 = FashionMNIST_NN(wandb.config.input_dim, wandb.config.output_dim, wandb.config.h_sizes, 
                          wandb.config.dprob, wandb.config.non_linearity, wandb.config.batch_norm)

# Initialize weights from normal distribution with mean 0 and standard deviation 0.01
def init_weights(layer):
  if type(layer) == nn.Linear:
    torch.nn.init.kaiming_normal_(layer.weight, mean = 0, std = 0.01)
    # torch.nn.init.normal_(layer.weight, mean = 0, std = 0.001)
    torch.nn.init.zeros_(layer.bias)

model2.to(wandb.config.device)
# model.apply(init_weights)

# loss_function
loss_function = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model2.parameters(), 
                            lr = wandb.config.learning_rate, 
                            weight_decay = wandb.config.weight_decay)

# scheduler = ReduceLROnPlateau(optimizer, mode='min', factor= wandb.config.scheduler_factor, 
#                              patience=wandb.config.scheduler_patience, verbose=True)

#scheduler = StepLR(optimizer, gamma=0.4,step_size=1, verbose=True)

In [ ]:
# See live graphs in the notebook.
#%%wandb 
batch_ct_train, batch_ct_valid = 0, 0
train_loss_history, train_acc_history, valid_loss_history, valid_acc_history = train_loop(train_loader, 
                                                                                          valid_loader, 
                                                                                          model, 
                                                                                          optimizer, 
                                                                                          loss_function, 
                                                                                          wandb.config.epochs, 
                                                                                          wandb.config.device,
                                                                                          wandb.config.patience,
                                                                                          wandb.config.early_stopping,
                                                                                          wandb.config.file_model,
                                                                                          )